In [1]:
import pandas as pd
import numpy as np
import math
import pickle
import collections

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import StratifiedKFold

import platform
from os import listdir
from os.path import isfile, join
from glob import glob
from pathlib import Path
import sys
import os
import copy
import traceback
from pathlib import Path


import matplotlib.pyplot as plt

import SMOTE
import feature_selector
import DE
import CFS
import birch
import metrics.abcd
import birch_bellwether

import metrices
import measures

import sys
import traceback
import warnings
warnings.filterwarnings("ignore")

In [2]:
def cluster_driver(df,print_tree = False):
    X = df.apply(pd.to_numeric)
    cluster = birch.birch(branching_factor=20)
        #X.set_index('Project Name',inplace=True)
    cluster.fit(X)
    cluster_tree,max_depth = cluster.get_cluster_tree()
        #cluster_tree = cluster.model_adder(cluster_tree)
    if print_tree:
        cluster.show_clutser_tree()
    return cluster,cluster_tree,max_depth

def build_BIRCH(attr_df):
    cluster,cluster_tree,_ = cluster_driver(attr_df)
    return cluster,cluster_tree

In [11]:
#get cluster wise data for summarzation using median
def find_bellwether(data_source1,data_source2,other_projects,path,fold):
    df_train = pd.read_pickle(data_source1 + '/train_data.pkl')
    cluster,cluster_tree = build_BIRCH(df_train)
    for _ in range(1):
        df = pd.read_csv(data_source1 + '/bellwether_cdom_0.csv')
        _cluster = {}
        for j in range(df.shape[0]):
            cluster_id = df.iloc[j,0]
            _bellwether = [df.iloc[j,7]]
            _cluster[cluster_id] = [_bellwether,list(df_train.iloc[cluster_tree[0].data_points].index)]
        bell = birch_bellwether.bellwether(path,df)
        for _bell_cluster in _cluster:
            #print(_cluster[_bell_cluster][0],_cluster[_bell_cluster][1])
            final_score = bell.bellwether(_cluster[_bell_cluster][0],_cluster[_bell_cluster][1])
            _results = {}
            goals = ['f1','g','recall','precision','pf']
            for goal in goals:
                for s_project in final_score.keys():
                    if s_project not in _results.keys():
                        _results[s_project] = {}
                    for d_projects in final_score[s_project].keys():
                        if goal == 'g':
                            _goal = 'g-score'
                        else:
                            _goal = goal
                        _results[s_project][d_projects] = np.median(final_score[s_project][d_projects][_goal])
                data_path = data_source2 + '/' + str(0)
                if not Path(data_path).is_dir():
                    os.makedirs(Path(data_path))
                _df_results = pd.DataFrame.from_dict(_results, orient='index')
                _df_results = _df_results.to_csv(data_path +'/bellwether_' + goal +'.csv')
                _df_results = pd.read_csv(data_path +'/bellwether_' + goal +'.csv')
        

In [12]:
for i in range(20):
    print(i)
    path = '/Users/suvodeepmajumder/Documents/AI4SE/bellwether_comminity/data/1385/converted'
    data_source1 = '/Users/suvodeepmajumder/Documents/AI4SE/bellwether_comminity/src/data/1385/exp2/2/fold_' + str(i)
    data_source2 = '/Users/suvodeepmajumder/Documents/AI4SE/bellwether_comminity/src/data/1385/exp2/0/cdom/fold_' + str(i)
    if platform.system() == 'Darwin' or platform.system() == 'Linux':
        _dir = path + '/'
    else:
        _dir = path + '\\'

    projects = [f for f in listdir(_dir) if isfile(join(_dir, f))]
    #print(projects)
    find_bellwether(data_source1,data_source2,projects,path,i)
    

0
benojt.csv
1
openmalaria.csv
2
lphant.csv
3
rectracker.csv
4
magicwars.csv
5
glas.csv
6
magicwars.csv
7
magicwars.csv
8
botsnscouts.csv
9
springside.csv
10
jitterbit.csv
11
owlib.csv
12
ldview.csv
13
lphant.csv
14
n2cms.csv
15
geogebra.csv
16
jmule.csv
17
jmule.csv
18
jmule.csv
19
twostep.csv
